In [30]:
import sys
import os
sys.path.append(os.path.abspath("../"))  # or "../../" depending on location

In [31]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from utils.load_models import set_device, get_CNN_small, get_CNN, get_resnet
import numpy as np
from tqdm import tqdm

In [32]:
# --- パラメータ設定 ---
batch_size = 128
epsilon = 1  # 摂動の最大幅
alpha = 0.01   # 学習率
num_epochs = 5
patch_size = (8, 8)  # 左上に貼るパッチのサイズ
device = set_device()

In [33]:

# ==========================
# 変換定義（正規化なし）
# ==========================
transform = transforms.ToTensor()
normalize = transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 std=(0.2023, 0.1994, 0.2010))

# ==========================
# CIFAR-10データ読み込み
# ==========================

# ダウンロード済みのデータを指定して読み込み
root = "../data/"
train_dataset = datasets.CIFAR10(
    root=root, 
    train=True, 
    download=False,  
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
subset_size = 10000
indices = np.random.choice(len(train_dataset), subset_size, replace=False)

# サブセットデータセットを作成
small_train_dataset = Subset(train_dataset, indices)
train_loader_small = DataLoader(small_train_dataset, batch_size=128, shuffle=True)


# --- パッチのマスク生成関数 ---
def generate_mask(size=(8, 8), image_shape=(3, 32, 32)):
    mask = torch.zeros(image_shape)
    mask[:, 0:size[0], 0:size[1]] = 1.0
    return mask

# --- マスクとパッチ初期化 ---
mask = generate_mask(size=patch_size).to(device)
mask = mask.unsqueeze(0)  # [1, 3, 32, 32]
delta = torch.zeros((1, 3, 32, 32), requires_grad=True, device=device)

def train_local_uap(
    model, 
    save_path,
    train_dataset=None,
    image_shape=(3, 32, 32),
    patch_size=(8, 8),
    epsilon=0.1,
    alpha=0.01,
    batch_size=128,
    num_epochs=5,
    device=None
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    model.eval()

    # --- データセットの準備（未指定ならCIFAR10を使う） ---
    if train_dataset is None:
        transform = transforms.ToTensor()
        train_dataset = datasets.CIFAR10(
            root="../data", train=True, download=True, transform=transform
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # --- マスクと摂動の初期化 ---
    mask = generate_mask(patch_size, image_shape).to(device).unsqueeze(0)  # [1, C, H, W]
    delta = torch.zeros((1, *image_shape), requires_grad=True, device=device)

    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            adv_images = (images + delta * mask).clamp(0, 1)
            outputs = model(adv_images)
            loss = F.cross_entropy(outputs, labels)

            loss.backward()
            with torch.no_grad():
                grad = delta.grad
                delta += alpha * torch.sign(grad) * mask
                delta.clamp_(-epsilon, epsilon)
                delta.grad.zero_()
            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

    # --- 保存 ---
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(delta.detach().cpu(), save_path)
    print(f"Local UAP saved to {save_path}")

    return delta.detach()
def train_local_uap_with_confidence_penalty(
    model,
    save_path,
    train_dataset=None,
    image_shape=(3, 32, 32),
    patch_size=(8, 8),
    epsilon=0.1,
    alpha=0.01,
    batch_size=128,
    num_epochs=5,
    lambda_conf=0.5,
    device=None,
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    model.eval()

    # --- データセット準備 ---
    if train_dataset is None:
        transform = transforms.ToTensor()
        train_dataset = datasets.CIFAR10(
            root="../data", train=True, download=True, transform=transform
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # --- マスクと摂動の初期化 ---
    def generate_mask(patch_size=(8, 8), image_shape=(3, 32, 32)):
        mask = torch.zeros(image_shape)
        h, w = patch_size
        mask[:, :h, :w] = 1.0
        return mask.unsqueeze(0).to(device)

    mask = generate_mask(patch_size, image_shape)
    delta = torch.zeros((1, *image_shape), requires_grad=True, device=device)

    # --- 学習ループ ---
    for epoch in range(num_epochs):
        total_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)
            adv_images = (images + delta * mask).clamp(0, 1)
            outputs = model(adv_images)

            ce_loss = F.cross_entropy(outputs, labels)
            probs = F.softmax(outputs, dim=1)
            correct_class_probs = probs[range(len(labels)), labels]
            conf_penalty = correct_class_probs.mean()

            loss = ce_loss - lambda_conf * conf_penalty  # ← 修正済み符号

            loss.backward()
            with torch.no_grad():
                grad = delta.grad
                delta += alpha * torch.sign(grad) * mask
                delta.clamp_(-epsilon, epsilon)
                delta.grad.zero_()
            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

    # --- 保存 ---
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(delta.detach().cpu(), save_path)
    print(f"Local UAP (with confidence penalty) saved to {save_path}")

    return delta.detach()

def train_local_uap_with_entropy_maximization(
    model,
    save_path,
    train_dataset=None,
    image_shape=(3, 32, 32),
    patch_size=(8, 8),
    epsilon=0.1,
    alpha=0.01,
    batch_size=128,
    num_epochs=5,
    lambda_entropy=1.0,
    device=None,
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    model.eval()

    # --- データセットの準備（未指定ならCIFAR10） ---
    if train_dataset is None:
        transform = transforms.ToTensor()
        train_dataset = datasets.CIFAR10(
            root="../data", train=True, download=True, transform=transform
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # --- マスクと摂動の初期化 ---
    def generate_mask(patch_size=(8, 8), image_shape=(3, 32, 32)):
        mask = torch.zeros(image_shape)
        h, w = patch_size
        mask[:, :h, :w] = 1.0
        return mask.unsqueeze(0).to(device)

    mask = generate_mask(patch_size, image_shape)
    delta = torch.zeros((1, *image_shape), requires_grad=True, device=device)

    # --- 学習ループ ---
    for epoch in range(num_epochs):
        total_loss = 0.0
        for images, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = images.to(device)
            adv_images = (images + delta * mask).clamp(0, 1)
            outputs = model(adv_images)

            # --- softmax entropy 最大化 ---
            probs = F.softmax(outputs, dim=1)
            entropy = -(probs * probs.log()).sum(dim=1).mean()

            # --- 損失（最大化したいのでそのまま） ---
            loss = lambda_entropy * entropy

            loss.backward()
            with torch.no_grad():
                grad = delta.grad
                delta += alpha * torch.sign(grad) * mask
                delta.clamp_(-epsilon, epsilon)
                delta.grad.zero_()
            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] Entropy Loss: {total_loss:.4f}")

    # --- 保存 ---
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(delta.detach().cpu(), save_path)
    print(f"Entropy-maximizing Local UAP saved to {save_path}")

    return delta.detach()

In [34]:
cnn = get_CNN()
cnn = cnn.to(device)
cnn_local_uap_path = "local_uap/local_uap_delta_CNN.pt"
train_local_uap(cnn,cnn_local_uap_path,train_dataset=train_dataset,epsilon=epsilon,device=device)

/Users/yamashitanatsuki/Documents/Python/portfolio_adverasrial_example/utils/load_models.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load

[Epoch 1] Loss: 3751.9282
[Epoch 2] Loss: 4253.8378
[Epoch 3] Loss: 4268.6321
[Epoch 4] Loss: 4270.2065
[Epoch 5] Loss: 4270.5661
Local UAP saved to local_uap/local_uap_delta_CNN.pt


tensor([[[[-1.0000,  0.7900, -0.0900,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0000, -0.3500, -0.3900,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0000, -0.1900,  0.2100,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.9800, -0.2700, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.8100,  0.1500,  0.1700,  ...,  0.0000,  0.0000,  0.0000],
          [-0.7800,  0.0700,  0.5500,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.5700, -1.0000, -0.6900,  ...,  0.0000,  0.0000,  0.0000],
          [-0.8800,  0.6900, -

In [35]:
cnn_small = get_CNN_small()
cnn_small = cnn_small.to(device)
cnn_small_local_uap_path = "local_uap/local_uap_delta_CNN_small.pt"
train_local_uap(cnn_small,cnn_small_local_uap_path,train_dataset=small_train_dataset,epsilon=epsilon,device=device)

/Users/yamashitanatsuki/Documents/Python/portfolio_adverasrial_example/utils/load_models.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.loa

[Epoch 1] Loss: 397.1076
[Epoch 2] Loss: 436.9184
[Epoch 3] Loss: 440.0053
[Epoch 4] Loss: 442.4949
[Epoch 5] Loss: 442.5877
Local UAP saved to local_uap/local_uap_delta_CNN_small.pt


tensor([[[[-0.6800, -0.9900, -0.2500,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.2500,  1.0000, -0.7900,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.6900,  0.9600,  0.4100,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000, -0.9900, -0.9900,  ...,  0.0000,  0.0000,  0.0000],
          [-0.9400,  0.1500, -0.4700,  ...,  0.0000,  0.0000,  0.0000],
          [-0.5100, -0.8300, -0.2900,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000,  0.9900,  0.9900,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.9000,  1.0000,  

In [36]:
cnn = get_CNN()
cnn = cnn.to(device)
cnn_local_uap_path_withConfidencePenalty = "local_uap/local_uap_delta_CNN_withConfidencePenalty.pt"
train_local_uap_with_confidence_penalty(
    cnn,
    cnn_local_uap_path_withConfidencePenalty,
    train_dataset=train_dataset,
    epsilon=epsilon,
    device=device
)


Epoch 1/5: 100%|██████████| 391/391 [00:08<00:00, 46.13it/s]


[Epoch 1] Loss: 3860.3103


Epoch 2/5: 100%|██████████| 391/391 [00:08<00:00, 47.89it/s]


[Epoch 2] Loss: 4351.0924


Epoch 3/5: 100%|██████████| 391/391 [00:08<00:00, 46.88it/s]


...

Epoch 4/5: 100%|██████████| 391/391 [00:08<00:00, 46.06it/s]


[Epoch 4] Loss: 4372.2564


Epoch 5/5: 100%|██████████| 391/391 [00:08<00:00, 46.97it/s]

[Epoch 5] Loss: 4372.1666
Local UAP (with confidence penalty) saved to local_uap/local_uap_delta_CNN_withConfidencePenalty.pt


tensor([[[[ 0.0300, -0.9800, -0.2500,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.4300, -0.4700, -0.9800,  ...,  0.0000,  0.0000,  0.0000],
          [-0.3700,  1.0000, -0.0900,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-1.0000, -1.0000, -0.0500,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000, -1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.5300,  0.8800, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-0.9700, -1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.8800, -1.0000, -

In [37]:
cnn_small = get_CNN_small()
cnn_small = cnn_small.to(device)
cnn_small_local_uap_path_withConfidencePenalty = "local_uap/local_uap_delta_CNN_small_withConfidencePenalty.pt"
train_local_uap_with_confidence_penalty(
    cnn_small,
    cnn_small_local_uap_path_withConfidencePenalty,
    train_dataset=small_train_dataset,
    epsilon=epsilon,
    device=device
)



Epoch 1/5: 100%|██████████| 79/79 [00:01<00:00, 44.76it/s]


[Epoch 1] Loss: 391.0393


Epoch 2/5: 100%|██████████| 79/79 [00:01<00:00, 49.01it/s]


[Epoch 2] Loss: 429.1241


Epoch 3/5: 100%|██████████| 79/79 [00:01<00:00, 44.53it/s]


...

Epoch 4/5: 100%|██████████| 79/79 [00:01<00:00, 48.02it/s]


[Epoch 4] Loss: 432.6491


Epoch 5/5: 100%|██████████| 79/79 [00:01<00:00, 45.56it/s]


[Epoch 5] Loss: 433.1467
Local UAP (with confidence penalty) saved to local_uap/local_uap_delta_CNN_small_withConfidencePenalty.pt


tensor([[[[-1.0000, -0.2900, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.2100, -0.1500,  0.1100,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.6100,  0.3100, -0.0300,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.9700, -1.0000, -0.3100,  ...,  0.0000,  0.0000,  0.0000],
          [-0.2700, -0.6300,  0.1900,  ...,  0.0000,  0.0000,  0.0000],
          [-0.1300, -0.4700, -0.3700,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000,  1.0000,  0.9800,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  1.0000,  

In [38]:
cnn_small_local_uap_path_withEntropy = "local_uap/local_uap_delta_CNN_small_withEntropy.pt"
train_local_uap_with_entropy_maximization(
    cnn_small,
    cnn_small_local_uap_path_withEntropy,
    train_dataset=small_train_dataset,
    epsilon=epsilon,
    device=device
)

Epoch 1/5: 100%|██████████| 79/79 [00:01<00:00, 58.82it/s]


[Epoch 1] Entropy Loss: 81.9725


Epoch 2/5: 100%|██████████| 79/79 [00:01<00:00, 57.77it/s]


[Epoch 2] Entropy Loss: 107.0222


Epoch 3/5: 100%|██████████| 79/79 [00:01<00:00, 56.69it/s]


...

Epoch 4/5: 100%|██████████| 79/79 [00:01<00:00, 57.84it/s]


[Epoch 4] Entropy Loss: 111.7099


Epoch 5/5: 100%|██████████| 79/79 [00:01<00:00, 60.94it/s]

[Epoch 5] Entropy Loss: 111.9693
Entropy-maximizing Local UAP saved to local_uap/local_uap_delta_CNN_small_withEntropy.pt


tensor([[[[ 0.7900, -0.8600,  0.9200,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.6900,  0.9800,  0.9700,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.1900,  0.7900, -0.7300,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.9100, -0.7100,  0.1700,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.8200,  0.7500,  0.3700,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.6100,  0.3500, -0.4300,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.4900, -0.9900, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0000,  0.4900, -

In [39]:
cnn_local_uap_path_withEntropy = "local_uap/local_uap_delta_CNN_withEntropy.pt"
train_local_uap_with_entropy_maximization(
    cnn,
    cnn_local_uap_path_withEntropy,
    train_dataset=train_dataset,
    epsilon=epsilon,
    device=device
)

Epoch 1/5: 100%|██████████| 391/391 [00:06<00:00, 59.42it/s]


[Epoch 1] Entropy Loss: 639.2249


Epoch 2/5: 100%|██████████| 391/391 [00:06<00:00, 60.20it/s]


[Epoch 2] Entropy Loss: 673.1503


Epoch 3/5: 100%|██████████| 391/391 [00:06<00:00, 61.01it/s]


...

Epoch 4/5: 100%|██████████| 391/391 [00:06<00:00, 61.27it/s]


[Epoch 4] Entropy Loss: 675.0379


Epoch 5/5: 100%|██████████| 391/391 [00:06<00:00, 60.72it/s]

[Epoch 5] Entropy Loss: 675.0308
Entropy-maximizing Local UAP saved to local_uap/local_uap_delta_CNN_withEntropy.pt


tensor([[[[-0.9200,  0.7500,  0.9800,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.9800,  0.8500, -0.3200,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.9800,  1.0000, -0.8500,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000, -0.9600, -0.9600,  ...,  0.0000,  0.0000,  0.0000],
          [-0.6500, -0.8300,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  0.6700,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-0.9600, -1.0000, -0.1200,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0000, -1.0000,  